In [11]:
import pandas as pd
import numpy as np
import tqdm as tqdm

In [1]:
books = pd.read_csv("books_with_categories.csv")

C:\Users\Rohit\AppData\Local\Temp\ipykernel_22808\1715236995.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
from transformers import pipeline
classifier = pipeline("text-classification", 
                      model="j-hartmann/emotion-english-distilroberta-base",
                      top_k = None,
                      device = "cuda"
                      return_all_scores=True)
classifier("I love this!")

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

D:\Jupyter\test\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Rohit\.cache\huggingface\hub\models--j-hartmann--emotion-english-distilroberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0
D:\Jupyter\test\lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'anger', 'score': 0.004419783595949411},
  {'label': 'disgust', 'score': 0.001611993182450533},
  {'label': 'fear', 'score': 0.00041385157965123653},
  {'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'neutral', 'score': 0.005764594301581383},
  {'label': 'sadness', 'score': 0.002092392183840275},
  {'label': 'surprise', 'score': 0.008528673090040684}]]

- we will run sentiment prediction over complete description and also at sentence level

In [3]:
classifier(books["description"][0])

[[{'label': 'anger', 'score': 0.003935152664780617},
  {'label': 'disgust', 'score': 0.019100716337561607},
  {'label': 'fear', 'score': 0.654841423034668},
  {'label': 'joy', 'score': 0.0151612414047122},
  {'label': 'neutral', 'score': 0.16985194385051727},
  {'label': 'sadness', 'score': 0.11640876531600952},
  {'label': 'surprise', 'score': 0.02070068195462227}]]

In [4]:
classifier(books["description"][0])

[[{'label': 'anger', 'score': 0.003935152664780617},
  {'label': 'disgust', 'score': 0.019100716337561607},
  {'label': 'fear', 'score': 0.654841423034668},
  {'label': 'joy', 'score': 0.0151612414047122},
  {'label': 'neutral', 'score': 0.16985194385051727},
  {'label': 'sadness', 'score': 0.11640876531600952},
  {'label': 'surprise', 'score': 0.02070068195462227}]]

In [10]:
#check for a particular sentence for peace of mind
sentences = books["description"][0].split(".")
predictions = classifier(sentences)
sentences[0], predictions[0]

('A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives',
 [{'label': 'anger', 'score': 0.00915635097771883},
  {'label': 'disgust', 'score': 0.002628469606861472},
  {'label': 'fear', 'score': 0.06816212832927704},
  {'label': 'joy', 'score': 0.04794234782457352},
  {'label': 'neutral', 'score': 0.14038528501987457},
  {'label': 'sadness', 'score': 0.002122158883139491},
  {'label': 'surprise', 'score': 0.7296032905578613}])

In [5]:
# way to create a dict such that for each sentence we have their emotion probability with us
emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [12]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [13]:
emotion_scores

{'anger': [0.06413363665342331,
  0.6126189827919006,
  0.06413363665342331,
  0.3514835834503174,
  0.08141224086284637,
  0.232224702835083,
  0.5381836295127869,
  0.06413363665342331,
  0.3006698787212372,
  0.06413363665342331],
 'disgust': [0.27359163761138916,
  0.3482847511768341,
  0.10400667041540146,
  0.15072251856327057,
  0.18449507653713226,
  0.7271749377250671,
  0.15585501492023468,
  0.10400667041540146,
  0.2794811427593231,
  0.1779269576072693],
 'fear': [0.928167998790741,
  0.9425276517868042,
  0.9723208546638489,
  0.3607066869735718,
  0.09504348784685135,
  0.051362816244363785,
  0.7474286556243896,
  0.4044956862926483,
  0.9155241250991821,
  0.051362816244363785],
 'joy': [0.9327973127365112,
  0.7044203877449036,
  0.7672369480133057,
  0.2518809139728546,
  0.04056441783905029,
  0.043375857174396515,
  0.8725652098655701,
  0.04056441783905029,
  0.04056441783905029,
  0.04056441783905029],
 'sadness': [0.6462163925170898,
  0.8879395127296448,
  0.54

In [14]:
from tqdm import tqdm

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}


for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████████████████████████████████████████████████████████████████████████| 5230/5230 [02:13<00:00, 39.07it/s]


In [18]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn
emotions_df.head()

,anger,disgust,fear,joy,sadness,surprise,neutral,isbn13
0,0.064134,0.273592,0.928168,0.932797,0.646216,0.967157,0.729603,9780002005883
1,0.612619,0.348285,0.942528,0.704420,0.887940,0.111690,0.252545,9780002261982
2,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078766,9780006178736
3,0.351484,0.150723,0.360707,0.251881,0.732686,0.111690,0.078766,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884390,0.475880,0.078766,9780006280934


In [17]:
books = pd.merge(books, emotions_df, on = "isbn13")


In [19]:
books.to_csv("books_with_emotions.csv", index=False)
